In [1]:
import sys

#!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework

In [2]:
TASK = 'MRPC' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

# Download glue data.
#!test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
#!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK

TASK_DATA_DIR = 'glue_data/' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

***** Task data directory: glue_data/MRPC *****
dev_ids.tsv  msr_paraphrase_test.txt   test.tsv
dev.tsv      msr_paraphrase_train.txt  train.tsv


In [3]:
BERT_MODEL = 'cased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

W0903 20:16:46.625719 140530993456960 deprecation_wrapper.py:119] From bert_repo/run_classifier_with_tfhub.py:151: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0903 20:16:47.102885 140530993456960 deprecation_wrapper.py:119] From bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 'token',
 '##izer']

In [89]:
MAX_SEQ_LENGTH = 128
processor = run_classifier.MrpcProcessor()
label_list = processor.get_labels()
prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:8]
input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)

In [87]:
MAX_SEQ_LENGTH = 128
processor = run_classifier.MrpcProcessor()
label_list = processor.get_labels()
prediction_examples = []
prediction_examples.append(run_classifier.InputExample(guid='###231232', text_a='Good evening', text_b='Good afternoon', label='1'))
input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)

In [92]:
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model import tag_constants
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], "/home/absin/Downloads/content/export_t/1566219509")
        tensor_input_ids = graph.get_tensor_by_name('input_ids_1:0')
        tensor_input_mask = graph.get_tensor_by_name('input_mask_1:0')
        tensor_label_ids = graph.get_tensor_by_name('label_ids_1:0')
        tensor_segment_ids = graph.get_tensor_by_name('segment_ids_1:0')
        tensor_outputs = graph.get_tensor_by_name('loss/Softmax:0')
        for i,input_feature in enumerate(input_features):
            input_ids = np.array(input_feature.input_ids).reshape(-1, 128)
            input_mask = np.array(input_feature.input_mask).reshape(-1, 128)
            label_ids = np.array(input_feature.label_id)
            segment_ids = np.array(input_feature.segment_ids).reshape(-1, 128)
            result = sess.run(tensor_outputs, feed_dict={
                    tensor_input_ids: input_ids,
                    tensor_input_mask: input_mask,
                    tensor_label_ids: [],
                    tensor_segment_ids: segment_ids,
                })
            print(prediction_examples[i].label)
            print(*(result[0]), sep='\t')
            print('-------')

1
0.0023771962	0.9976228
-------
0
0.9963199	0.0036800979
-------
0
0.14532445	0.85467553
-------
1
0.002536964	0.997463
-------
0
0.99533373	0.0046662455
-------
1
0.002593101	0.9974069
-------
0
0.0033952503	0.9966048
-------
1
0.0024598227	0.99754024
-------


In [94]:
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model import tag_constants
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], "/home/absin/Downloads/content/export_t/1566219509")
        tensor_input_ids = graph.get_tensor_by_name('input_ids_1:0')
        tensor_input_mask = graph.get_tensor_by_name('input_mask_1:0')
        tensor_label_ids = graph.get_tensor_by_name('label_ids_1:0')
        tensor_segment_ids = graph.get_tensor_by_name('segment_ids_1:0')
        tensor_outputs = graph.get_tensor_by_name('loss/Softmax:0')
        record_iterator = tf.python_io.tf_record_iterator(path="/home/absin/git/bert_repo/tf_examples_1.tfrecord")
        for string_record in record_iterator:
            example = tf.train.Example()
            example.ParseFromString(string_record)
            input_ids = example.features.feature['input_ids'].int64_list.value
            input_mask = example.features.feature['input_mask'].int64_list.value
            label_ids = example.features.feature['label_ids'].int64_list.value
            segment_ids = example.features.feature['segment_ids'].int64_list.value
            print(np.array(segment_ids).reshape(-1, 128).shape)
            result = sess.run(tensor_outputs, feed_dict={
                    tensor_input_ids: np.array(input_ids).reshape(-1, 128),
                    tensor_input_mask: np.array(input_mask).reshape(-1, 128),
                    tensor_label_ids: np.array(label_ids),
                    tensor_segment_ids: np.array(segment_ids).reshape(-1, 128),
                })
            print(*(result[0]), sep='\t')
            print('-------')

(1, 128)
0.0024648812	0.9975351
-------
(1, 128)
0.002424499	0.99757546
-------
(1, 128)
0.002491034	0.997509
-------
(1, 128)
0.0025251713	0.99747485
-------
(1, 128)
0.0024491025	0.9975509
-------
